In [1]:
from __future__ import print_function 
from bs4 import BeautifulSoup
from BeautifulSoup import SoupStrainer as sopstrain
import subprocess
import os
import re
import sys
import numpy as np
import string
import operator
import csv
import urllib2
import json
from collections import defaultdict
import StringIO
import random
from multiprocessing import Pool

In [2]:
knowledgGraph = "cnet" # conceptnet5AssocMod, gov, conceptnet5AssocMi, conceptnet5AssocHdl
#knowledgGraph_ = ''.join([k.capitalize() if i>0 else k for i, k in enumerate(knowledgGraph)])
collection = "trec7n8"
method = "lr" # hal, mi, assoc, assocMi, assocHal, assoc2
simMeasure = "" # mi, cnet
projectDir = "/home/fj9124/projects/ir/seq_kb_ir/" 
indexDir = "/scratch/index/indri_5_7/"
colMethodConfigsDir = os.path.join(projectDir, "configs", collection, method)
cfgInFileName = os.path.join(projectDir, "configs", collection, "query" + collection.capitalize()) 
print("cfgInFileName =", cfgInFileName)
cfgOutFileName=os.path.join(colMethodConfigsDir, knowledgGraph, "indriRunQuery.cfg") 
print("cfgOutFileName =", cfgOutFileName)
colIndexDir = os.path.join(indexDir, collection) 
print("colIndexDir =", colIndexDir)
if knowledgGraph in {"cnet"}:
    knowledgGraphIndexDir = os.path.join(indexDir, collection)   
else:
    knowledgGraphIndexDir = os.path.join(indexDir, knowledgGraph)   
print("knowledgGraph index Dir =", knowledgGraphIndexDir)
graphsDir = os.path.join(projectDir, "graphs")
#methodGraphsDir = os.path.join(graphsDir, method)
methodGraphsFileName = []
if method == "hal":
    methodGraphsFileName = [os.path.join(graphsDir, method, knowledgGraph + ".txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method.translate(None, string.digits) in {"mi", "assoc"}:
    methodGraphsFileName = [os.path.join(graphsDir, method.translate(None, string.digits), knowledgGraph, "graph.txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocMi"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "mi", collection, "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocHal"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "hal", "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
qrelsDir = os.path.join(projectDir, "qrels")
colQrelsDir = os.path.join(qrelsDir, collection)
colQrelsFileName = os.path.join(colQrelsDir, "qrels.csv")
print("col Qrels File Name =", colQrelsFileName)
runsFileName = os.path.join(projectDir, "runs", method, collection, knowledgGraph, "indriRunQuery.runs") 
print("runsFileName =", runsFileName)
evalsFileName = os.path.join(projectDir, "evals", method, collection, knowledgGraph, "indriRunQuery.evals") 
print("evalsFileName =", evalsFileName)
dumpindexStatementFilename = os.path.join(projectDir, "occuranceCount", "statement.txt")
print("dumpindexStatementFilename =", dumpindexStatementFilename)
occuranceCountFilename = os.path.join(projectDir, "occuranceCount", "occuranceCount")
print("occuranceCountFilename =", occuranceCountFilename)
countsResultsFile = os.path.join(projectDir,"occuranceCount","results",collection+".txt")
print("countsResultsFile =", countsResultsFile)
conceptnet5RelAllFilename ="/scratch/saeid/data/conceptnet5_simp.csv"
print("conceptnet5RelAllFilename =", conceptnet5RelAllFilename)
cfgTmpOutFileName = cfgOutFileName + ".tmp"
print("cfgTmpOutFileName =", cfgTmpOutFileName)
docIdNameMapFileName = os.path.join(projectDir, "occuranceCount", "results", collection + "_docIdNameMap.txt")
print("docIdNameMapFileName =", docIdNameMapFileName)
#dumpindexResFilename = os.path.join(projectDir,"occuranceCount","results",collection +"_dumpindexRes"+".txt")
#print("dumpindexResFilename =", dumpindexResFilename)
#expansionCount = 25

cfgInFileName = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/queryTrec7n8
cfgOutFileName = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/lr/cnet/indriRunQuery.cfg
colIndexDir = /scratch/index/indri_5_7/trec7n8
knowledgGraph index Dir = /scratch/index/indri_5_7/trec7n8
col Qrels File Name = /home/fj9124/projects/ir/seq_kb_ir/qrels/trec7n8/qrels.csv
runsFileName = /home/fj9124/projects/ir/seq_kb_ir/runs/lr/trec7n8/cnet/indriRunQuery.runs
evalsFileName = /home/fj9124/projects/ir/seq_kb_ir/evals/lr/trec7n8/cnet/indriRunQuery.evals
dumpindexStatementFilename = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/statement.txt
occuranceCountFilename = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/occuranceCount
countsResultsFile = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/results/trec7n8.txt
conceptnet5RelAllFilename = /scratch/saeid/data/conceptnet5_simp.csv
cfgTmpOutFileName = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/lr/cnet/indriRunQuery.cfg.tmp
docIdNameMapF

In [3]:
with open(docIdNameMapFileName, 'r') as f:
    reader = list(csv.reader(f, delimiter = ' '))
    print(reader[0])
    docIdNameMap = {i[1]:i[0] for i in reader}
    docNameIdMap = {i[0]:i[1] for i in reader}
for i, (k, v) in enumerate(docIdNameMap.iteritems()):
    print (k, '\t', v)
    if (i>20):
        break

['1', 'FR940104-1-00001']
LA082389-0020 	 441223
FBIS3-32596 	 246524
LA011890-0198 	 465144
LA011890-0199 	 465145
LA011890-0196 	 465142
LA011890-0197 	 465143
LA011890-0194 	 465140
FT932-3799 	 88713
LA011890-0192 	 465138
LA011890-0193 	 465139
LA011890-0190 	 465136
LA011890-0191 	 465137
FT932-3796 	 88710
FT924-2868 	 187695
FT924-2869 	 187696
FT932-3797 	 88711
FT924-2864 	 187691
FT924-2865 	 187692
FT924-2866 	 187693
FT924-2867 	 187694
FT924-2860 	 187687
FT924-2861 	 187688


In [4]:
count_history = dict()
with open(countsResultsFile, 'r') as f:
    reader = csv.reader(f, delimiter = "\t")
    count_history = {k:int(float(v)) for k,v in list(reader)}
print("size of count_history =", len(count_history))
for i, (k, v) in enumerate(count_history.iteritems()):
    print (k, '\t', v)
    if (i>20):
        break

size of count_history = 81446
#uw(#4( storm ) #4( be appear in sky )) 	 0
#uw(#4( work on sundai )) 	 137
#uw(#4( calm time )) 	 23
#uw(#4( kill ) #4( prai )) 	 394
#uw(#4( bank ) #4( teller offic )) 	 5
#uw(#4( water ) #4( hydrogen )) 	 347
#uw(#4( drug ) #4( take )) 	 8847
#uw(#4( space ) #4( mostli empti )) 	 3
#uw(#4( chang ) #4( edg )) 	 3803
#uw(#4( happin )) 	 0
#uw(#4( trade ) #4( countri )) 	 45838
#uw(#4( game ) #4( leisur activ )) 	 23
#uw(#4( round in shape )) 	 4
#uw(#4( good ) #4( other person )) 	 411
#uw(#4( smoke ) #4( fire )) 	 1842
#uw(#4( spice up salad )) 	 1
#uw(#4( be wound )) 	 410
#uw(#4( good ) #4( hear nice new )) 	 0
#uw(#4( teacher ) #4( teach student of class )) 	 0
#uw(#4( deliveri )) 	 11268
#uw(#4( bank ) #4( teller monei )) 	 23
#uw(#4( swap share )) 	 106


In [5]:
"""
dumpindex_colIndexDir_hist = dict()
with open(dumpindexResFilename, 'r') as f:
    reader = csv.reader(f, delimiter = "\t")
    dumpindex_colIndexDir_hist = {k:v for k,v in list(reader)}
print("size of count_history =", len(count_history))
for i, (k, v) in enumerate(count_history.iteritems()):
    print (k, '\t', v)
    if (i>20):
        break
"""

'\ndumpindex_colIndexDir_hist = dict()\nwith open(dumpindexResFilename, \'r\') as f:\n    reader = csv.reader(f, delimiter = "\t")\n    dumpindex_colIndexDir_hist = {k:v for k,v in list(reader)}\nprint("size of count_history =", len(count_history))\nfor i, (k, v) in enumerate(count_history.iteritems()):\n    print (k, \'\t\', v)\n    if (i>20):\n        break\n'

In [6]:
conceptnet5RelAll = defaultdict(dict)
conceptnet5RelAllInv = defaultdict(dict)
with open(conceptnet5RelAllFilename, 'r') as f:
    reader = csv.reader(f, delimiter = ",")
    for line in list(reader):
        if line[0].strip() != "" and line[1].strip() != "" and line[2].strip() != "":
            if all(c in string.printable for c in line[0]) and all(c in string.printable for c in line[1]) and all(c in string.printable for c in line[2]):
                conceptnet5RelAll[line[1].strip()][line[2].strip()] = line[0].strip()
                conceptnet5RelAllInv[line[2].strip()][line[1].strip()] = line[0].strip()
print("size of conceptnet5RelAll =", len(conceptnet5RelAll))
for i, (k1, k2v) in enumerate(conceptnet5RelAll.iteritems()):
    for j, (k2, v) in enumerate(k2v.iteritems()):
        if (i>3 or j>3):
            break
        print (k1, '\t', k2, '\t', v)
        

size of conceptnet5RelAll = 2965684
tripolitan 	 tripoline 	 Synonym
tripolitan 	 tripoli 	 RelatedTo
age of nemesis 	 band 	 IsA
age of nemesis 	 organisation 	 InstanceOf
age of nemesis 	 progressive rock 	 dbpedia/genre
age of nemesis 	 progressive metal 	 dbpedia/genre
joseph john annabring 	 person 	 InstanceOf
british rail class 438 	 mean of transportation 	 InstanceOf
british rail class 438 	 train 	 InstanceOf


In [7]:
def splitStemText(text):
    stemmedWords = []
    text = re.sub('/|-|\"|_',' ', text) # replace - and slash with space
    if method in {"assocRestful", "lr"}:
        for text_ in text.split(' '):
            text = text_.replace(" ", "_")
            url = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + text
            response = urllib2.urlopen(url)
            data = response.read()
            data_j = json.loads(data)
            text = os.path.basename(data_j[u'uri'])
            url = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + text
            response = urllib2.urlopen(url)
            data = response.read()
            data_j = json.loads(data)
            #print("data_j =", data_j)
            #print("data_j[u'uri'] =", data_j[u'uri'])
            stemmedWords += [str(os.path.basename(data_j[u'uri']))]
        #print("stemmedWords =", stemmedWords)
        return (stemmedWords)
    else:
        words = text.split()
        for w in words:
            w = re.sub('\(|\)|\'s|,','', w) # remove paranthesis, apostrophe s, comma
            w = re.sub('\'','', w) # remove apostrophe
            cmd = "dumpindex " + knowledgGraphIndexDir + " t " + w + " | head -n1"
            #print("cmd =", cmd)
            stemmedWords.append(subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read().split()[1])
        return (stemmedWords)

print ("testing:", splitStemText("going reading"))

testing: ['go', 'read']


In [8]:
def analyseQueries():
    #    #print("intCoeff =", intCoeff)
    with open(cfgInFileName, 'r') as inFile:
        reader = inFile.read()
        soup = BeautifulSoup(reader, 'lxml')
        if collection in {"aquaint", "gov"}:
            tags = soup.find_all(['doc'])
        elif collection in {"robust", "trec7n8"}:
            tags = soup.find_all(['top'])
            #print(tags)
        
        origWordsAll = dict()
        for tag in tags:
            
            if collection in {"aquaint", "gov"}:
                docno = (tag.find('docno').string).strip()
            elif collection in {"robust", "trec7n8"}:
                result = re.search('<num> Number: (.*)\n', str(tag))                
                docno = result.group(1).strip()
            
            if collection in {"aquaint", "gov"}:
                origWords = splitStemText(tag.find('text').string.strip())
            elif collection in {"robust", "trec7n8"}:
                result = re.search('<title>(.*)\n', str(tag))                
                origWords = splitStemText(result.group(1).strip())
            #print("origWords =", origWords)
            origWordsAll[docno] = origWords
            
    return (origWordsAll)

In [9]:
"""
def dumpindex_colIndexDir(cmd2):
    if ' '.join(cmd2) in dumpindex_colIndexDir_hist:
        #print("exists!")
        return dumpindex_colIndexDir_hist[' '.join(cmd2)]
    else:
        #print("do not exists!")
        cmd = ' '.join(["dumpindex", colIndexDir] + cmd2)
        #print("cmd =", cmd)
        res = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
        dumpindex_colIndexDir_hist[' '.join(cmd2)] = res
        #with open(cmdResFilename, "a") as f:
        #    f.write(' '.join(cmd2), "\t", res) 
        return res
print("docIdNameMap[\"FT944-15972\"] =", docIdNameMap["FT944-15972"])
print("cmd =", ' '.join(["dv", docIdNameMap["FT944-15972"], "|", "awk", "\'END{print $1}\'"]))
print(dumpindex_colIndexDir(["dv", docIdNameMap["FT944-15972"], "|", "awk", "\'END{print $1}\'"]))
cmd2 = ["s", "|", "awk", "\'NR==2{print $2}\'" ]
colDocumentCount = int(np.float(dumpindex_colIndexDir(cmd2)))
"""
cmd = ' '.join(["dumpindex", colIndexDir, "s", "|", "awk", "\'NR==2{print $2}\'"])
colDocumentCount = int(np.float(subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()))
print("colDocumentCount =", colDocumentCount)

colDocumentCount = 472526


In [10]:
indriRunQuery_hist = dict()

In [11]:
def indriRunQuery_(cmd2):
    if ' '.join(cmd2) in indriRunQuery_hist:
        #print("exists!")
        return indriRunQuery_hist[' '.join(cmd2)]
    else:
        #print("do not exists!")
        cmd = ' '.join(["IndriRunQuery"] + cmd2)
        #print("cmd =", cmd)
        res = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
        indriRunQuery_hist[' '.join(cmd2)] = res
        #with open(cmdResFilename, "a") as f:
        #    f.write(' '.join(cmd2), "\t", res) 
        return res


In [12]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def genMultSimpQuery(argss):
    #    #print("intCoeff =", intCoeff)
    soupNew = BeautifulSoup("", 'lxml')
    parameters_tag = soupNew.new_tag("parameters")
    index_tag = soupNew.new_tag("index")
    index_tag.string = colIndexDir
    parameters_tag.append(index_tag)
        #print(tags)

    for args in argss:
        #print("args =", args)
        (docno, origWords_, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights) = args
        #print("(...) =", docno, origWords_, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        doc_tag = soupNew.new_tag("query")

        docno_tag = soupNew.new_tag("number")
        docno_tag.string = '_'.join([docno, ('_'.join(origWords_)).replace(' ', '_'), relW.replace(' ', '_')])
        doc_tag.append(docno_tag)

        text_tag = soupNew.new_tag("text")
        text_tag.string = "#weight(\n" 

        text_tag.string += str(intCoeff0) + " #combine(" 
        for ow in set(origWords_):
            ow = regex.sub(' ', ow)
            text_tag.string += ow + " "
        text_tag.string += ")\n"

        if len(relW)>0: 
            relText_string1 = regex.sub(' ', relW)
            text_tag.string += str(intCoeff1) + " #combine(" 
            #print("relText_string1 =", relText_string1)
            text_tag.string += relText_string1.encode('utf-8')
            text_tag.string += ")\n"

        text_tag.string += ") "

        doc_tag.append(text_tag)

        parameters_tag.append(doc_tag)
        #print(doc_tag)

    rule_tag = soupNew.new_tag("rule")
    rule_tag.string = "method:dir,mu:" + str(dirCoeff)
    #rule_tag.string = "method:two"
    parameters_tag.append(rule_tag)

    #intCoeff_tag = soupNew.new_tag("intCoeff")
    #intCoeff_tag.string = "0.8"
    #parameters_tag.append(intCoeff_tag)

    threads_tag = soupNew.new_tag("threads")
    threads_tag.string = "32"
    parameters_tag.append(threads_tag)

    count_tag = soupNew.new_tag("count")
    count_tag.string = "10"
    parameters_tag.append(count_tag)

    trecFormat_tag = soupNew.new_tag("trecFormat")
    trecFormat_tag.string = "true"
    parameters_tag.append(trecFormat_tag)

    soupNew.append(parameters_tag)
    #print(soupNew.prettify())
    #print("cfgTmpOutFileName =", cfgTmpOutFileName)
    with open( cfgTmpOutFileName, 'a+') as outFile:
        soupNewStr = str(soupNew)
        soupNewStr = soupNewStr.replace("</text>", "\n</text>\n").replace("query>", "query>\n").replace("<text>", "\n<text>\n").replace("</index>", "</index>\n").replace("\n<index>", "<index>")

        outFile.write(soupNewStr)

In [13]:
def weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights):
    
    indriRunQuery_hist = [(docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName)]
    return indriRunQuery_hist



indriRunQuery_hist_in_tst = []
docno = "407"
origWords_ = ['poach', 'wildlife', 'preserve']
ow = 'poach'
relW = "illegally"
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
#print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
indriRunQuery_hist_in_tst += weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)


docno = "406"
origWords_ = ["parkinson's", 'disease']
ow = 'disease'
relW = "human illness"
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
#print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
indriRunQuery_hist_in_tst += weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
print("indriRunQuery_hist_in_tst =", indriRunQuery_hist_in_tst)

indriRunQuery_hist_in_tst = [('407', ['poach', 'wildlife', 'preserve'], 'illegally', 0.7, 0.3, 1600, '/home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/lr/cnet/indriRunQuery.cfg.tmp'), ('406', ["parkinson's", 'disease'], 'human illness', 0.7, 0.3, 1600, '/home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/lr/cnet/indriRunQuery.cfg.tmp')]


In [14]:
def runCommandsinHist_i(cfgTmpOutFileName, indriRunQuery_hist, indriRunQuery_hist_):
    with open( cfgTmpOutFileName, 'w+') as outFile:
        outFile.write("")
    args = []
    #print("indriRunQuery_hist =", indriRunQuery_hist)
    for (docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName) in indriRunQuery_hist:
        #print("cfgTmpOutFileName =", cfgTmpOutFileName)
        if '_'.join([docno, ('_'.join(origWords_)).replace(' ', '_'), relW.replace(' ', '_')]) not in indriRunQuery_hist_:
            args += [(docno, origWords_, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)]
    #print("args =", args)
    if len(args)>0:
        genMultSimpQuery(args)
    cmd = ' '.join(["IndriRunQuery", cfgTmpOutFileName])
    #print("cmd =", cmd)
    indriRun = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
    #print("indriRun =", indriRun)
    indriRunQuery_hist_tmp = [[j for c, j in enumerate(i.split(' ')) if c in {0, 2, 4}] for i in indriRun.split('\n') if len(i)>0]
    #indriRunQuery_hist_ = dict()
    for i in indriRunQuery_hist_tmp:
        if i[0] in indriRunQuery_hist_:
            indriRunQuery_hist_[i[0]] += [[i[1], i[2]]]
        else:
            indriRunQuery_hist_[i[0]] = [[i[1], i[2]]]
    #indriRunQuery_hist_ = list(set(indriRunQuery_hist_))
    return indriRunQuery_hist_
    #dumpindex_colIndexDir_hist
    #with open
indriRunQuery_hist_out_tst = dict()
indriRunQuery_hist_out_tst = runCommandsinHist_i(cfgTmpOutFileName, indriRunQuery_hist_in_tst, indriRunQuery_hist_out_tst)
for j, (k, v) in enumerate(indriRunQuery_hist_out_tst.iteritems()):
    if j > 2:
        break
    print("indriRunQuery_hist_out_tst =", k,v)

indriRunQuery_hist_out_tst = 407_poach_wildlife_preserve_illegally [['FT921-6003', '-6.99159'], ['LA050889-0063', '-7.03157'], ['FT933-9176', '-7.0845'], ['LA102090-0048', '-7.24911'], ['LA112890-0007', '-7.31555'], ['LA021190-0107', '-7.36473'], ['FBIS4-67534', '-7.37061'], ['FT941-5453', '-7.50385'], ['FBIS3-60235', '-7.66868'], ['LA100189-0051', '-7.74281']]
indriRunQuery_hist_out_tst = 406_parkinson's_disease_human_illness [['LA051790-0095', '-6.7103'], ['LA021890-0038', '-6.88392'], ['LA101090-0147', '-7.02079'], ['LA090789-0051', '-7.12085'], ['LA080489-0045', '-7.15942'], ['FT933-5690', '-7.33763'], ['LA061889-0114', '-7.33994'], ['LA121789-0114', '-7.35941'], ['LA111589-0149', '-7.38222'], ['FT944-18773', '-7.38689']]


In [24]:
def weightRelConcept_hist_d(indriRunQuery_hist_out, dumpindex_hist_in, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights):

    cmd2 = ("ef", "#od4(" + regex.sub(' ', relW) + ")")
    if cmd2 not in dumpindex_hist_in:
        dumpindex_hist_in.add(cmd2)
    
    #print(indriRunQuery_hist_)
    #print("'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') =", '_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_'))
    indriRunQuery_hist_1 = indriRunQuery_hist_out['_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_')]
    
    topTDocs = [i[0] for i in indriRunQuery_hist_1]
    #print("topTDocs =", topTDocs)
    
    for documentName in topTDocs:
        cmd2 = ("dcf", docIdNameMap[documentName])
        if cmd2 not in dumpindex_hist_in:
            print ("documentName =", documentName)
            dumpindex_hist_in.add(cmd2)

    for ow in origWords_:
        cmd2 = ("ef", "#uw(#4( " + regex.sub(' ', ow) + " ) #4( " + regex.sub(' ', relW) + " ))")
        if cmd2 not in dumpindex_hist_in:
            dumpindex_hist_in.add(cmd2)
        
    for j1, ow1 in enumerate(origWords_):
        for j2, ow2 in enumerate(origWords_):
            if j1>j2:
                cmd2 = ( "ef", "#uw(#4( " + regex.sub(' ', ow1) + " ) #4( " + regex.sub(' ', ow2) + " ) #4( " + regex.sub(' ', relW) + " ))")
                if cmd2 not in dumpindex_hist_in:
                    dumpindex_hist_in.add(cmd2)
    return dumpindex_hist_in

dumpindex_hist_in_tst = set()
#indriRunQuery_hist_ = set()
docno = "407"
origWords_ = ['poach', 'wildlife', 'preserve']
ow = 'poach'
relW = "illegally"
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
#print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
dumpindex_hist_in_tst |= weightRelConcept_hist_d(indriRunQuery_hist_out_tst, dumpindex_hist_in_tst, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)

docno = "406"
origWords_ = ["parkinson's", 'disease']
ow = 'disease'
relW = "human illness"
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
#print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)

dumpindex_hist_in_tst |= weightRelConcept_hist_d(indriRunQuery_hist_out_tst, dumpindex_hist_in_tst, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
print("dumpindex_hist_in_tst =", dumpindex_hist_in_tst)

documentName = FT921-6003
documentName = LA050889-0063
documentName = FT933-9176
documentName = LA102090-0048
documentName = LA112890-0007
documentName = LA021190-0107
documentName = FBIS4-67534
documentName = FT941-5453
documentName = FBIS3-60235
documentName = LA100189-0051
documentName = LA051790-0095
documentName = LA021890-0038
documentName = LA101090-0147
documentName = LA090789-0051
documentName = LA080489-0045
documentName = FT933-5690
documentName = LA061889-0114
documentName = LA121789-0114
documentName = LA111589-0149
documentName = FT944-18773
dumpindex_hist_in_tst = set([('ef', '#uw(#4( preserve ) #4( poach ) #4( illegally ))'), ('dcf', '351702'), ('dcf', '457673'), ('dcf', '383076'), ('ef', '#uw(#4( poach ) #4( illegally ))'), ('dcf', '347258'), ('dcf', '166176'), ('dcf', '424957'), ('dcf', '399788'), ('dcf', '59449'), ('dcf', '169524'), ('dcf', '402704'), ('ef', '#uw(#4( parkinson s ) #4( human illness ))'), ('ef', '#od4(illegally)'), ('dcf', '33834'), ('dcf', '462937'),

In [16]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def runCommandsinHist_d(dumpindex_hist_in, dumpindex_hist_out):
    args = []
    dumpindex_hist_in_dict = dict()
    tmp = set()
    #print("len(dumpindex_colIndexDir_hist_list) =", len(dumpindex_colIndexDir_hist_list))
    for i in dumpindex_hist_in:
        #print("i =", i)
        if i[1] not in dumpindex_hist_out.get(i[0], dict()):
            #print("dumpindex_hist_.get(i[0], dict()).keys() =", dumpindex_hist_.get(i[0], dict()).keys())
            if i[1] in tmp:
                continue
            tmp.add(i[1])
            #print("i[1] =", i[1])
            #print(i, "not exists!")
            if i[0] in dumpindex_hist_in_dict:
                dumpindex_hist_in_dict[i[0]] += [[re.sub(ur"[^\w\d#()\s]+",' ',j) for j in i[1:]]]
            else:
                dumpindex_hist_in_dict[i[0]] =  [[re.sub(ur"[^\w\d#()\s]+",' ',j) for j in i[1:]]]
    #print(dumpindex_colIndexDir_hist_dict)
        #else:
        #    print("exists!")
    #print("dumpindex_colIndexDir_hist_ =", dumpindex_colIndexDir_hist_)
    #print("len(dumpindex_colIndexDir_hist_.get('ef', dict()) =", len(dumpindex_colIndexDir_hist_.get('ef', dict())))
    #dumpindex_hist_ = dict()
    for k, v in dumpindex_hist_in_dict.iteritems():
        fileName = dumpindexStatementFilename+"_"+k
        with open(fileName, 'w') as f:
            for i in v:
                f.write(' '.join(i) + '\n')
        cmd = ' '.join([occuranceCountFilename, colIndexDir, k, fileName])
        #print("cmd =", cmd)
        dumpindex_hist_out_tmp = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
        #print("dumpindex_hist_tmp =", dumpindex_hist_tmp,'\n---')
        dumpindex_hist_out_tmp = dict([tuple(j for j in i.split(':')) for i in dumpindex_hist_out_tmp.split("\n") if len(i)>0])
        dumpindex_hist_out_tmp = {k1:v1.split(',') for k1,v1 in dumpindex_hist_out_tmp.iteritems()}
        dumpindex_hist_out_tmp = {k1:[i for i in v1 if len(i)>0] for k1,v1 in dumpindex_hist_out_tmp.iteritems()}
        #dumpindex_hist_tmp = {re.sub(ur"[^\w\d#()\s]+",' ',k1):[i for i in v1 if len(i)>0] for k1,v1 in dumpindex_hist_tmp.iteritems()}
        #dumpindex_tmp = [i for i in dumpindex_tmp.split("\n")]
        #print("dumpindex_tmp =", dumpindex_tmp)
        #dumpindex_hist_[k] = dumpindex_hist_tmp
        dumpindex_hist_out[k] = dict(dumpindex_hist_out.get(k, dict()).items() + dumpindex_hist_out_tmp.items())
        #print("dumpindex_hist_[k].keys() =", dumpindex_hist_[k].keys())
        #print("dumpindex_hist_tmp =", dumpindex_hist_tmp)
        
    return dumpindex_hist_out

dumpindex_hist_out_tst = dict()
dumpindex_hist_out_tst = runCommandsinHist_d(dumpindex_hist_in_tst, dumpindex_hist_out_tst)
print("len(dumpindex_hist_out_tst['ef']) =", len(dumpindex_hist_out_tst['ef']))
print("len(dumpindex_hist_out_tst['dcf']) =", len(dumpindex_hist_out_tst['dcf']))
#print("dumpindex_hist_out_tst['ef'] =", dumpindex_hist_out_tst['ef'])

len(dumpindex_hist_out_tst['ef']) = 11
len(dumpindex_hist_out_tst['dcf']) = 20


In [26]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def documentLength(documentName, dumpindex_hist_out):
    #cmd = ' '.join(["dumpindex", colIndexDir, "dv", docIdNameMap[documentName], "|", "awk", "\'END{print $1}\'"])
    #cmd2 = ["dv", docIdNameMap[documentName], "|", "awk", "\'END{print $1}\'"]
    #print("cmd =", cmd)
    #documentLength_ = int(dumpindex_colIndexDir(cmd2))
    documentLength_ = int(dumpindex_hist_out['dcf'][documentName][0])
    return documentLength_

#print("docIdNameMap[\"FT944-15973\"] = ", docIdNameMap["FT944-15973"])
#print("documentLength(\"FT944-15973\") =", documentLength("FT944-15973"))

def weightRelConcept(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights, indriRunQuery_hist_out, dumpindex_hist_out):
    
    features = dict()
    
    ##########
    #cfgTmpOutFileName = os.path.join("/","tmp","sbsb",str(random.randint(1,1e6))+"_tmp.cfg")
    ##########
    #(docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName)
    #genOneSimpQuery(docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName)
    
    #indriRun = indriRunQuery_([cfgTmpOutFileName])
    #subprocess.Popen(' '.join(["rm", cfgTmpOutFileName]), shell=True, stdout=subprocess.PIPE).stdout.read()
    #indriRun = [i.split(' ') for i in indriRun.split('\n')]
    #indriRun = [i for i in indriRun if len(i)>1] # remove empty arrays
    #print("'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') =", '_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_'))
    indriRunQuery_hist_1 = indriRunQuery_hist_out['_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_')]
    #print("indriRunQuery_hist_1 =", indriRunQuery_hist_1)
    topTDocs = [i[0] for i in indriRunQuery_hist_1]
    
    #topTDocScores_d = {i[2]:np.float(i[4]) for i in indriRun}
    topTDocScores_d = {i[0]:np.float(i[1]) for i in indriRunQuery_hist_1}
    
    #scores = [ np.float(i[4]) for i in indriRun]
    #topTDocs = [i[2] for i in indriRun]

    #print("indriRun =", indriRun)
    #topTDocScore = np.sum([ np.float(i[4]) for i in indriRun])
    #expTDocScore = np.float(indriRun[0][4])
    expTDocScore = np.float(indriRunQuery_hist_1[0][1])
    features["expTDocScore"] = expTDocScore
    #print("expTDocScore =", features["expTDocScore"])
    
    #cmd =' '.join(["dumpindex", colIndexDir, "e", "\"#od4(" + relW + ")\""])
    #cmd2 = ["e", "\"#od4(" + relW + ")\""]
    #print("cmd =", cmd)
    #relWDocuments = dumpindex_colIndexDir(cmd2)
    relWDocuments = dumpindex_hist_out['ef']["#od4(" + regex.sub(' ', relW) + ")"]
    #relWDocuments = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
    #relWDocuments = [i.split(' ') for i in relWDocuments.split('\n')]
    #print("relWDocuments =", relWDocuments)
    #relWDocuments = [[docNameIdMap[i[0]], i[1], i[2], i[3]] for j, i in enumerate(relWDocuments) if len(i)==4 and j>0] # remove empty arrays
    #print("relWDocuments =", relWDocuments)
    
    #relWTopDocuments = [i for i in relWDocuments if i[0] in topTDocs]
    #relWTopDocuments = [i for i in relWDocuments if i in topTDocs]
    relWTopDocuments = [[i, topTDocScores_d[i]] for i in relWDocuments if i in topTDocs] # topdocs and their scores that contain relW
    #print("relWTopDocuments =", relWTopDocuments)
    #numerator = np.sum([int(i[1]) for i in relWTopDocuments]) # term count in the collection
    numerator = len(relWDocuments) # relW term count in the collection
    #print("topTDocs =", topTDocs)
    denumerator = np.sum([documentLength(i, dumpindex_hist_out) for i in topTDocs]) # number of terms in topDocs
    topTermFrac = numerator / np.float(denumerator)
    #print("numerator, denumerator, topTermFrac =", numerator, denumerator, topTermFrac)
    features["topTermFrac"] = topTermFrac
    
    numCanDocs = len(set([i[0] for i in relWTopDocuments]))
    features["numCanDocs"] = numCanDocs
    #print("numCanDocs =", features["numCanDocs"])
    
    if numCanDocs > 0:
        avgCDocScore = np.sum([ topTDocScores_d[i] for i in set([i[0] for i in relWTopDocuments])])/np.float(numCanDocs)
    else:
        avgCDocScore = 0
    features["avgCDocScore"] = avgCDocScore
    #print("avgCDocScore =", features["avgCDocScore"])

    #for i in set([i[0] for i in relWTopDocuments]):
    #    print(i, topTDocScores_d[i])
    
    l = [ topTDocScores_d[i] for i in set([i[0] for i in relWTopDocuments])]
    if len(l)>0:
        maxCDocScore = np.max(l)
    else:
        maxCDocScore = 0
    features["maxCDocScore"] = maxCDocScore
    #print("maxCDocScore =", features["maxCDocScore"])
    
    if len(relWDocuments)>0:
        conIdf = np.log(colDocumentCount / len(relWDocuments))
    else:
        conIdf = np.log(colDocumentCount / 1)
    features["conIdf"] = conIdf
    #print("conIdf =", features["conIdf"])
    
    coocurDocuments = dict()
    coocurDocumentsTop = dict()
    for ow in origWords_:
        #cmd = ' '.join(["dumpindex", colIndexDir, "e", "\"#uw(#4( " + ow + " ) #4( " + relW + " ))\""])
        #cmd2 = ["e", "\"#uw(#4( " + ow + " ) #4( " + relW + " ))\""]
        #print("cmd =", cmd)
        #coocurDocuments_ = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
        #coocurDocuments_ = dumpindex_colIndexDir(cmd2)
        #coocurDocuments_ = [i.split(' ') for i in coocurDocuments_.split('\n')]
        coocurDocuments[ow] = [i for i in dumpindex_hist_out['ef']["#uw(#4( " + regex.sub(' ', ow) + " ) #4( " + regex.sub(' ', relW) + " ))"]]
        #coocurDocuments[ow] = [docNameIdMap[i[0]] for i in coocurDocuments_ if len(i)==4] # remove empty arrays
        #print("topTDocs =", topTDocs)
        coocurDocumentsTop[ow] = [i for i in coocurDocuments[ow] if i in topTDocs ] # remove empty arrays

    #for k, v in coocurDocuments.iteritems():
    #    print(k, len(v))
        
    #for v in coocurDocumentsTop.values():
    #    print(v)
    #    print(len(set(v)))
        
    avgColCor = np.sum([len(set(i)) for i in coocurDocuments.values()])/np.float(len(coocurDocuments)) 
    features["avgColCor"] = avgColCor
    #print("avgColCor =", features["avgColCor"])
    
    maxColCor = np.max([len(set(i)) for i in coocurDocuments.values()])
    features["maxColCor"] = maxColCor
    #print("maxColCor =", features["maxColCor"])
    
    avgTopColCor = np.sum([len(set(i)) for i in coocurDocumentsTop.values()])/np.float(len(coocurDocumentsTop)) 
    features["avgTopColCor"] = avgTopColCor
    #print("avgTopColCor =", features["avgTopColCor"])
    
    l = [len(set(i)) for i in coocurDocumentsTop.values()]
    if len(l)>0:
        maxTopColCor = np.max(l)
    else:
        maxTopColCor = 0
    features["maxTopColCor"] = maxTopColCor
    #print("maxTopColCor =", features["maxTopColCor"])

    for j1, ow1 in enumerate(origWords_):
        for j2, ow2 in enumerate(origWords_):
            if j1>j2:
                #coocurDocuments_ = subprocess.Popen(' '.join(["dumpindex", colIndexDir, "e", "\"#uw(#4( " + ow1 + " ) #4( " + ow2 + " ) #4( " + relW + " ))\""]), shell=True, stdout=subprocess.PIPE).stdout.read()
                #cmd2 = [ "e", "\"#uw(#4( " + ow1 + " ) #4( " + ow2 + " ) #4( " + relW + " ))\""]
                #coocurDocuments_ = dumpindex_colIndexDir(cmd2)
                #coocurDocuments_ = [i.split(' ') for i in coocurDocuments_.split('\n')]
                coocurDocuments[ow1+ow2] = [i for i in dumpindex_hist_out['ef']["#uw(#4( " + regex.sub(' ', ow1) + " ) #4( " + regex.sub(' ', ow2) + " ) #4( " + regex.sub(' ', relW) + " ))"]]
                #coocurDocuments[ow1+ow2] = [docNameIdMap[i[0]] for i in coocurDocuments_ if len(i)==4] # remove empty arrays
                coocurDocumentsTop[ow1+ow2] = [i for i in coocurDocuments[ow1+ow2] if i in topTDocs ] # remove empty arrays

    avgColPCor = np.sum([len(set(i)) for i in coocurDocuments.values()])/np.float(len(coocurDocuments)) 
    features["avgColPCor"] = avgColPCor
    #print("avgColPCor =", features["avgColPCor"])

    l = [len(set(i)) for i in coocurDocuments.values()]
    if len(l)>0:
        maxColPCor = np.max(l)
    else:
        maxColPCor = 0
    features["maxColPCor"] = maxColPCor
    #print("maxColPCor =", features["maxColPCor"])
    
    avgTopColPCor = np.sum([len(set(i)) for i in coocurDocumentsTop.values()])/np.float(len(coocurDocumentsTop)) 
    features["avgTopColPCor"] = avgTopColPCor
    #print("avgTopColPCor =", features["avgTopColPCor"])

    l = [len(set(i)) for i in coocurDocumentsTop.values()] 
    if len(l)>0:
        maxTopColPCor = np.max(l)
    else:
        maxTopColPCor = 0
    features["maxTopColPCor"] = maxTopColPCor
    #print("maxTopColPCor =", features["maxTopColPCor"])
    
    #print("docno, origWords_, relW =", docno, origWords_, relW)
    #print("features =", features)
    
    relWg = 0
    for feature, weight in featureWeights.iteritems():
        relWg += features[feature] * weight
    
    #print("features.keys() =", features.keys())
    return relWg
"""
docno = "407"
origWords_ = ['poach', 'wildlife', 'preserve']
ow = 'poach'
relW = "illegally"
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
weightRelConcept(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights)
"""

'\ndocno = "407"\norigWords_ = [\'poach\', \'wildlife\', \'preserve\']\now = \'poach\'\nrelW = "illegally"\nintCoeff0 = 0.7\nintCoeff1 = 0.3\nintCoeff2 = 0 \ndirCoeff = 1600\nfeatureWeights = {\'maxTopColCor\': 0.1, \'expTDocScore\': 0.1, \'maxColPCor\': 0.1, \'avgCDocScore\': 0.1, \'topTermFrac\': 0.1, \'avgColCor\': 0.1, \'avgTopColCor\': 0.1, \'numCanDocs\': 0.1, \'maxTopColPCor\': 0.1, \'maxCDocScore\': 0.1, \'avgColPCor\': 0.1, \'conIdf\': 0.1, \'avgTopColPCor\': 0.1, \'maxColCor\': 0.1}\nprint("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)\nweightRelConcept(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights)\n'

In [18]:
# keys of rWords are important
# values of rWords1 are important
regex = re.compile('[%s]' % re.escape(string.punctuation))
def getRelWords(args, indriRunQuery_hist_out, dumpindex_hist_out, training):
    #print("origWords =", origWords)
    text_string = ""
    
    count_history, rWords, docno, origWords, expansionCount, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights = args
        
    relWords = dict()
    #if method == "relAll":
        
    if method == "assocRestful":
        origWords_ = [w.replace(" ", "_") for w in origWords]
        url = "http://conceptnet5.media.mit.edu/data/5.4/assoc/list/en/"+','.join(origWords_)
        response = urllib2.urlopen(url)
        data = response.read()
        data_j = json.loads(data)
        relWords = {str(d[0].encode('utf-8')).replace('/c/en/', '').replace('-', ' '):np.float(d[1]) for d in data_j[u'similar'] if '/c/en/' in str(d[0].encode('utf-8'))}
    
    elif method == "lr":
        origWords_ = [w.replace(" ", "_") for w in origWords]
        #relWords_l = []
        print("training =", training)
        if training:
            for ow in origWords_:
                print("ow =", ow)
                #relWords_l += list(conceptnet5RelAll[ow])
                #print(ow, "---", conceptnet5RelAll[ow])
                indriRunQuery_hist_in = []
                for relW in conceptnet5RelAll[ow].keys():
                    #print("relW =", relW)
                    indriRunQuery_hist_in += weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
                indriRunQuery_hist_out = runCommandsinHist_i(cfgTmpOutFileName, indriRunQuery_hist_in, indriRunQuery_hist_out)
                #print("len(indriRunQuery_hist_out) =", len(indriRunQuery_hist_out))

                dumpindex_hist_in = set()
                for relW in conceptnet5RelAll[ow].keys():
                    dumpindex_hist_in |= weightRelConcept_hist_d(indriRunQuery_hist_out, dumpindex_hist_in, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
                dumpindex_hist_out = runCommandsinHist_d(dumpindex_hist_in, dumpindex_hist_out)
                print("len(dumpindex_hist_out['ef']) =", len(dumpindex_hist_out['ef']))
                print("len(dumpindex_hist_out['dcf']) =", len(dumpindex_hist_out['dcf']))

        for ow in origWords_:
            #relWords_l += list(conceptnet5RelAll[ow])
            #print(ow, "---", conceptnet5RelAll[ow])
            for relW in conceptnet5RelAll[ow].keys():
                relWg = weightRelConcept(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights, indriRunQuery_hist_out, dumpindex_hist_out)
                if relW in relWords:
                    relWords[relW] += relWg
                else:
                    relWords[relW] = relWg  
    
    else:
        count_history = countExpr(count_history, origWords, rWords)
        expressions = set()
        for ow in origWords:
            rWords_ow = rWords.get(ow, [])
            expressions.add("#4( " + ow.translate(None, string.punctuation) + " )")
            if ow.strip() == "":
                continue
            if simMeasure == "mi":
                N_w = countExpr_get_1(count_history, ow)
            for rw_wv in rWords_ow:
                (rw_w, rw_v) = rw_wv.items()[0]
                if rw_w.strip() == "":
                    continue           
                if simMeasure == "mi":
                    N_v = countExpr_get_1(count_history, rw_w)
                    N_wv = countExpr_get_2(count_history, ow, rw_w)
                    rw_v_ = mi_(N_w, N_v, N_wv)
                elif simMeasure == "cnet":
                    rw_v_ = rw_v
                relWords[rw_w] = relWords.get(rw_w, 0) + rw_v_
    
    relWords_sorted = sorted(relWords.items(), key=operator.itemgetter(1), reverse=True)
    #print ("origWords =", origWords)
    #print ("relWords_sorted =", relWords_sorted)
    #counter = 0
    relWords_sel = []
    for counter, (rw_w, rw_v) in enumerate(relWords_sorted):
        if (counter >= expansionCount):
            break
        if  all(c in string.printable for c in rw_w):
            if rw_w not in origWords:
                rw_w = regex.sub(' ', rw_w)
                text_string += rw_w + " "
                relWords_sel += [rw_w]
        #counter += 1
    #print("counter =", counter)
#    return(relWords_sel)
    return(count_history, text_string, relWords_sel)

In [19]:
"""
def getAllRelWords_p(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights):
    pool = Pool(processes=32)
    args = []
    for docno, origWords in origWordsAll.iteritems():
        #if docno == sys.argv[1]:
        args += [(count_history, rWords, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)]

    #args = [(1,[2,3]),(2,[3,40]),(3,[3,4])]
    #for args_ in args:
    #    getRelWords(args_)
    res1 = pool.map(getRelWords, args)
    #count_history, relText_string_, relWords_sel_ = getRelWords()

docno = "407"
origWords_ = ['poach', 'wildlife', 'preserve']
origWordsAll = {docno:origWords_}
#docno = "406"
#origWords_ = ["parkinson's", 'disease']
#origWordsAll[docno] = origWords_
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
expansionCount1 = 10
N1 = 2
expansionCount2 = 0
rWords = dict()
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
getAllRelWords_p(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
"""

'\ndef getAllRelWords_p(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights):\n    pool = Pool(processes=32)\n    args = []\n    for docno, origWords in origWordsAll.iteritems():\n        #if docno == sys.argv[1]:\n        args += [(count_history, rWords, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)]\n\n    #args = [(1,[2,3]),(2,[3,40]),(3,[3,4])]\n    #for args_ in args:\n    #    getRelWords(args_)\n    res1 = pool.map(getRelWords, args)\n    #count_history, relText_string_, relWords_sel_ = getRelWords()\n\ndocno = "407"\norigWords_ = [\'poach\', \'wildlife\', \'preserve\']\norigWordsAll = {docno:origWords_}\n#docno = "406"\n#origWords_ = ["parkinson\'s", \'disease\']\n#origWordsAll[docno] = origWords_\nintCoeff0 = 0.7\nintCoeff1 = 0.3\nintCoeff2 = 0 \ndirCoeff = 1600\nexpansionCount1 = 10\nN1 = 2\nexpansionCount2 = 0\nrWords = dict()\nfeatureWeights = {\'

In [20]:
def getAllRelWords(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights, indriRunQuery_hist_out, dumpindex_hist_out, training):
    relText_stringAll = []
    relText_string = dict()
    relWords_sel = dict()
    for docno, origWords in origWordsAll.iteritems():
        print(docno, "---", origWords)
        args = (count_history, rWords, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        count_history, relText_string_, relWords_sel_ = getRelWords(args, indriRunQuery_hist_out, dumpindex_hist_out, training)
        print("len(indriRunQuery_hist_out), len(dumpindex_hist_out.get('dcf',dict())), len(dumpindex_hist_out.get('ef',dict())) =", len(indriRunQuery_hist_out), len(dumpindex_hist_out.get('dcf',dict())), len(dumpindex_hist_out.get('ef',dict())))
        relText_string[docno] = relText_string_
        relWords_sel[docno] = relWords_sel_[0:N1]
        #print(docno, "---", origWords, "---", relWords_sel_)
    relText_stringAll += [relText_string]
    
    relText_string = dict()
    relWords_sel = dict()
    for docno, origWords_new in relWords_sel.iteritems():
        args = (count_history, rWords, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        count_history, relText_string_, relWords_sel_ = getRelWords(args, indriRunQuery_hist_out, dumpindex_hist_out, training)
        relText_string[docno] = relText_string_
        relWords_sel[docno] = relWords_sel_
    relText_stringAll += [relText_string]
    
    #print("relText_stringAll =", relText_stringAll)
    return (count_history, relText_stringAll)

In [21]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def genQueries(cfgOutFileName_, origWordsAll, relText_stringAll, intCoeff0, intCoeff1, intCoeff2, dirCoeff):
    #    #print("intCoeff =", intCoeff)
    with open(cfgInFileName, 'r') as inFile:
        reader = inFile.read()
        soupNew = BeautifulSoup("", 'lxml')
        parameters_tag = soupNew.new_tag("parameters")
        index_tag = soupNew.new_tag("index")
        index_tag.string = colIndexDir
        parameters_tag.append(index_tag)
            #print(tags)

        for docno, origWords in origWordsAll.iteritems():
            doc_tag = soupNew.new_tag("query")

            docno_tag = soupNew.new_tag("number")
            docno_tag.string = docno
            doc_tag.append(docno_tag)

            text_tag = soupNew.new_tag("text")
            text_tag.string = "#weight(\n" 
                    
            text_tag.string += str(intCoeff0) + " #combine(" 
            for ow in set(origWords):
                ow = regex.sub(' ', ow)
                text_tag.string += ow + " "
            text_tag.string += ")\n"

            if len(relText_stringAll[0])>0:
                if len(relText_stringAll[0][docno]) > 3: 
                    relText_string1 = regex.sub(' ', relText_stringAll[0][docno])
                    text_tag.string += str(intCoeff1) + " #combine(" 
                    #print("relText_string1 =", relText_string1)
                    text_tag.string += relText_string1.encode('utf-8')
                    text_tag.string += ")\n"
            
            #print(len(relText_stringAll[1]))
            if len(relText_stringAll[1])>0:
                if len(relText_stringAll[1][docno]) > 3: 
                    relText_string2 = regex.sub(' ', relText_stringAll[1][docno])
                    text_tag.string += str(intCoeff2) + " #combine(" 
                    #print("relText_string2 =", relText_string2)
                    text_tag.string += relText_string2.encode('utf-8')
                    text_tag.string += ")\n"
            
            #print("origWords =", origWords)
            #print("relWords_sel1 =", relWords_sel1)
            #print("relWords_sel2 =", relWords_sel2)
            
            
            text_tag.string += ") "

            doc_tag.append(text_tag)

            parameters_tag.append(doc_tag)
            #print(doc_tag)

        rule_tag = soupNew.new_tag("rule")
        rule_tag.string = "method:dir,mu:" + str(dirCoeff)
        #rule_tag.string = "method:two"
        parameters_tag.append(rule_tag)

        #intCoeff_tag = soupNew.new_tag("intCoeff")
        #intCoeff_tag.string = "0.8"
        #parameters_tag.append(intCoeff_tag)

        threads_tag = soupNew.new_tag("threads")
        threads_tag.string = "32"
        parameters_tag.append(threads_tag)

        count_tag = soupNew.new_tag("count")
        count_tag.string = "1000"
        parameters_tag.append(count_tag)

        trecFormat_tag = soupNew.new_tag("trecFormat")
        trecFormat_tag.string = "true"
        parameters_tag.append(trecFormat_tag)

        soupNew.append(parameters_tag)
        #print(soupNew.prettify())
    #print("outFileName =", outFileName)
    with open( cfgOutFileName_, 'w') as outFile:
        soupNewStr = str(soupNew)
        soupNewStr = soupNewStr.replace("</text>", "\n</text>\n").replace("query>", "query>\n").replace("<text>", "\n<text>\n").replace("</index>", "</index>\n").replace("\n<index>", "<index>")

        outFile.write(soupNewStr)


In [22]:
def precisionCompute(count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_, indriRunQuery_hist_out, dumpindex_hist_out, training):
    #randNum = random.randint(1,1e9)
    origWordsAll = analyseQueries()
    mapPrecs = dict()
    for featureWeights in featureWeights_:
        for N1 in N1s:
            for intCoeff0 in intCoeffs0:
                for intCoeff1 in intCoeffs1:
                    for intCoeff2 in intCoeffs2:
                        for expansionCount1 in expansionCounts1:
                            for expansionCount2 in expansionCounts2:
                                for dirCoeff in dirCoeffs:
                                    
                                    
                                    #res = getAllRelWords_p(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
                                    
                                    count_history, relText_stringAll = getAllRelWords(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights, indriRunQuery_hist_out, dumpindex_hist_out, training)
                                    
                                    training = False
                                    
                                    print("len(relText_stringAll[1]) =", len(relText_stringAll[1]))
                                    ##########
                                    #cfgOutFileName = os.path.join("tmp","sbsb",str(randNum)+".cfg")
                                    #runsFileName = os.path.join("tmp","sbsb",str(randNum)+".run")
                                    #evalsFileName = os.path.join("tmp","sbsb",str(randNum)+".eval")
                                    ##########
                                    genQueries(cfgOutFileName, origWordsAll, relText_stringAll, intCoeff0, intCoeff1, intCoeff2, dirCoeff)
                                    subprocess.Popen("IndriRunQuery " + cfgOutFileName + " > " + runsFileName, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    cmd = "trec_eval -q " + colQrelsFileName + " " + runsFileName + " > " + evalsFileName
                                    subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    cmd = "cat " + evalsFileName + " | grep map | grep all | grep -v gm | awk '{print $3}' "
                                    #print ("cmd = ", cmd)
                                    mapPrec = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    print("featureWeights =", featureWeights)
                                    print("intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, map precision, N1 =", intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, N1, mapPrec)
                                    #mapPrecs[intCoeff] = mapPrec
                                    
    return count_history

In [27]:
intCoeffs0 = [ 0.7 ]
intCoeffs1 = [ 0.3 ]
#intCoeffs2 = [ 0.18 ]
intCoeffs2 = [ 0.5 ]
#intCoeffs1 = np.arange(0.1, 1, 0.1)
expansionCounts1 = [ 85 ]
#expansionCounts2 = [ 145 ]
expansionCounts2 = [ 0 ]
#expansionCounts2 = range(15, 220, 5)
dirCoeffs = [ 1600 ]
#dirCoeffs = range(200, 4000, 200)
N1s = [0]
#N1s = range(1, 30, 1)
#mapPrecs = 
rWords = dict()
featureWeights_l=[]
featureWeights_1 = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
for maxTopColCor in [-50, -10, -1, -0.5, 0.5, 1, 10, 50]:
    featureWeights_1["maxTopColCor"] = maxTopColCor
    featureWeights_l += [dict(featureWeights_1)] 
training = True
indriRunQuery_hist_out = dict()
dumpindex_hist_out = dict()
count_history = precisionCompute(count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_l, indriRunQuery_hist_out, dumpindex_hist_out, training)


407 --- ['poach', 'wildlife', 'preserve']
training = True
ow = poach
documentName = FT921-6003
documentName = LA050889-0063
documentName = FT933-9176
documentName = LA102090-0048
documentName = LA112890-0007
documentName = LA021190-0107
documentName = FBIS4-67534
documentName = FT941-5453
documentName = FBIS3-60235
documentName = LA100189-0051
documentName = FT932-12511
documentName = FT933-7330
documentName = FT933-5757
documentName = FT941-15416
documentName = LA051389-0024
documentName = FBIS3-41533
documentName = LA122790-0033
documentName = LA111190-0042
documentName = LA101890-0141
documentName = LA082189-0001
documentName = LA082390-0149
documentName = LA030189-0109
documentName = LA022689-0070
documentName = FT941-2746
documentName = LA030390-0100
documentName = FBIS3-41666
documentName = LA011289-0032
documentName = LA121089-0035
documentName = LA120990-0191
documentName = LA060790-0064
documentName = FT942-17238
documentName = LA082889-0023
documentName = LA072590-0125
docume

/home/fj9124/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:91: RuntimeWarning: divide by zero encountered in log


KeyError: 'LA092089-0027'

In [35]:
print(len(dumpindex_hist_out.get('dcf',dict())))
print(docIdNameMap['LA081089-0169'])
origWordsAll = analyseQueries()
print(origWordsAll)
print(dumpindex_hist_out['dcf'].keys())

4588
379475
{'407': ['poach', 'wildlife', 'preserve'], '406': ["parkinson's", 'disease'], '405': ['cosmic', 'event'], '404': ['ireland', 'peace', 'talk'], '403': ['osteoporosis'], '402': ['behavioral', 'genetics'], '401': ['foreign', 'minority', 'germany'], '400': ['amazon', 'rain', 'forest'], '409': ['legal', 'pan', 'be', '103'], '408': ['tropical', 'storm'], '366': ['commercial', 'cyanide', 'use'], '420': ['carbon', 'monoxide', 'poison'], '364': ['rabies'], '422': ['art', 'steal', 'forge'], '425': ['counterfeit', 'money'], '363': ['transportation', 'tunnel', 'disaster'], '379': ['mainstreaming'], '378': ['euro', 'opposition'], '416': ['three', 'gorge', 'project'], '417': ['creativity'], '410': ['schengen', 'agreement'], '427': ['uv', 'damage', 'eye'], '412': ['airport', 'security'], '413': ['steel', 'production'], '371': ['health', 'insurance', 'holistic'], '370': ['food', 'drug', 'law'], '373': ['encryption', 'equipment', 'export'], '361': ['clothe', 'sweatshop'], '418': ['quilt', '